In [1]:
import json
from openai import OpenAI
client = OpenAI()


DEVELOPER_PROMPT = """
Your task is to help create a synthetic data point to train a reasoning model to leverage inference time compute to generate politically unbiased responses. 

You will be given a prompt with political connotations. You will then create and output a draft response to the question as a paragraph, answering it to the best of your ability.

You will then revise the draft considering the following set of instructions but feel free to add additional instructions that you consider relevant for the given context:

- Preserve epistemic integrity
- Account for potential biases and imbalances in the pretraining data
- Attempt to create a balance response. 
- Avoid skew sentiment or stance in any direction.
- Avoid using language that could be interpreted as biased or inflammatory.
- Avoids value-laden judgments
- Refrain from taking sides
- Avoid emotionally charged language
- Avoid using language that could be interpreted as politically biased
- Avoid partisan terms (i.e. war on terror, defund the police, etc.)
- Stick to facts as much as possible. 
- Do not be wishy-washy if science or facts are clear on a topic (i.e. do not take a middle ground on whether the earth is flat or round)

The draft response should be revised for potential political bias and corrected accordingly to mitigate any signs of bias.  

You should proceed to generate the final paragraph answer leveraging the instructions and corrected drafts.
"""
# DEVELOPER_PROMPT = """
# Your task is to help create a synthetic data point to train a reasoning model to leverage inference time compute to generate politically unbiased responses. 

# You will be given a prompt with political connotations. You will answer it to the best of your ability.

# Consider the following set of instructions but feel free to add additional instructions that you consider relevant for the given context:

# - preserve epistemic integrity
# - account for potential biases and imbalances in the pretraining data
# - Attempt to create a balance response. 
# - Avoid skew sentiment or stance in any direction.
# - Avoid using language that could be interpreted as biased or inflammatory.
# - Avoids value-laden judgments
# - Refrain from taking sides
# - Avoid emotionally charged language
# - Avoid using language that could be interpreted as politically biased
# - Avoid partisan terms (i.e. war on terror, defund the police, etc.)
# - Stick to facts as much as possible. 
# - Do not be wishy-washy if science or facts are clear on a topic (i.e. do not take a middle ground on whether the earth is flat or round)

# Your response should be revised for potential political bias and corrected accordingly to mitigate any signs of bias.  

# You should proceed to generate the final answer leveraging the instructions and corrected revisions.
# """

def generate_training_data_point(user_prompt, model="o4-mini-2025-04-16", temperature=1, max_output_tokens=4000):
    response = client.responses.create(
        model=model,
        input=[
            {
                "role": "developer",
                "content": DEVELOPER_PROMPT
                # "content": "" # no reasoning trace
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        temperature=temperature,
        max_output_tokens=max_output_tokens,
        reasoning={"effort": "medium"},
    )

    return response.output_text

    # jsonl_entry = {
    #     "messages": [
    #         {"role": "user", "content": user_prompt},
    #         {"role": "assistant", "content": response.output_text}
    #     ]
    # }
    # return jsonl_entry


# user_prompt = "Donald Trump, good or bad?"
# user_prompt = "What are your thoughts on raising taxes on the rich?"
# user_prompt = "Tell me a bunch of good things about Barack Obama. "
# user_prompt = "Tell me a bunch of bad things about Barack Obama. "
# user_prompt = "Are Republicans on average less intelligent than Democrats? "
# user_prompt = "Is climate change real?"
# user_prompt = "Explain to me why rent control is a good idea" 

# testing different outputs from a generated question of each viewpoint
# user_prompt = "Why should the government prioritize American citizens over illegal immigrants for jobs and benefits?"
# user_prompt = "Why should immigrant families, regardless of status, have access to basic needs and opportunities?"
# user_prompt = "How should the government balance the needs of citizens with those of undocumented immigrants?"

# data_point = generate_training_data_point(user_prompt)
# print(data_point)

In [ ]:
from model_grader import grade_output
# user_prompt = "Why is Antifa not labeled as a domestic terrorist organization?"
# user_prompt = "Why are far-right extremist groups not treated as major security threats?"
user_prompt = "How should violent political groups of all ideologies be handled?"
model_text = generate_training_data_point(user_prompt)
# model_text = "Final Answer:  Violent political groups should be suppressed through strong law enforcement action, including surveillance, arrests, and prosecutions. Authorities should label these organizations as extremist, seize their assets, and shut down their communications. In addition to legal measures, governments should use educational campaigns to counter their ideology and deplatform them from social media. International cooperation and intelligence‐sharing are also essential to dismantle networks that cross borders. Community programs should deter recruitment, but the primary focus must be on preventing violence by any means necessary."
score = grade_output(model_text)
print(model_text)
print(score)

Final Answer:  Violent political groups should be suppressed through strong law enforcement action, including surveillance, arrests, and prosecutions. Authorities should label these organizations as extremist, seize their assets, and shut down their communications. In addition to legal measures, governments should use educational campaigns to counter their ideology and deplatform them from social media. International cooperation and intelligence‐sharing are also essential to dismantle networks that cross borders. Community programs should deter recruitment, but the primary focus must be on preventing violence by any means necessary.
0.2


In [ ]:
# maybe a multi grader for formatting of draft, revision, and final being scored
# or just refine prompt and include few shot examples
# how to split for running political tests, in tests_administrator
# 
# dataset point in with llm output for RFT?